---  
title: "軌道最適化に関するノートブック"  
author: "Naoya Ozaki"  
date: "24 January 2024"  
output: "trajectory_optimization"  

---  

# Julia言語による軌道最適化に関するノートブック

## 1. SPICEの使い方

本章では，NASAジェット推進研究所(JPL)のNavigation and Ancillary Information Facility (NAIF)が提供するSPICE 1)の使い方をまとめる．C,FOTRAN, MATLAB等のインタフェースが提供されているが，ここではJuliaインタフェースである`SPICE.jl` 2)を用いて，惑星の精密な軌道情報を取得する．

1. https://naif.jpl.nasa.gov/naif/
2. https://github.com/JuliaAstro/SPICE.jl

In [1]:
import SPICE

# Load generic kernels
SPICE.furnsh("/Users/naoya/Programming/SPICE/generic_kernels/lsk/naif0012.tls") # Leap seconds kernel
SPICE.furnsh("/Users/naoya/Programming/SPICE/generic_kernels/pck/gm_de431.tpc") # Gravity Constant
SPICE.furnsh("/Users/naoya/Programming/SPICE/generic_kernels/spk/planets/de440.bsp") # Planetary ephemeris kernel

# Convert the calendar date to ephemeris seconds past J2000
et = SPICE.utc2et("2024-01-24T17:00:00")

# Get the position of Mars at `et` w.r.t. Earth
SPICE.spkezr("MOON", et, "J2000", "NONE", "EARTH")

([-149729.32542123165, 323380.9514241508, 178003.47653310827, -0.930096599343289, -0.31477987020879544, -0.13948942418980217], 1.3287384459430396)

## 2. `DifferentialEquations.jl`による軌道伝播

In [4]:
using DifferentialEquations
using LinearAlgebra

"""
   eom_nbp!(dxdt, x, p, t)

Computes the equation of motion for the N-body problem.

# Arguments
- `dxdt`: The derivative of the state vector.
- `x`: The state vector.
- `p`: The parameter sets.
- `t`: The time.
"""
function eom_nbp!(dxdt, x, p, t)
    # Initialize dx/dt
    dxdt[1:3] .= x[4:6]
    dxdt[4:6] .= p.u

    # Calculate Gravitational Acceleration
    for id in p.list_bodies
        # Get planetary ephemeris
        x_body = SPICE.spkez(id, t, "J2000", "NONE", 0)[1]
        r_body = x_body[1:3]
        #v_body = x_body[4:6] / (p.lsf / p.tsf)
        gm_body = SPICE.bodvcd(id, "GM", 1)[1]

        # Acceleration due to the body id
        r_rel = x[1:3] - r_body
        dxdt[4:6] .-= gm_body * r_rel / norm(r_rel)^3
    end
end

# Propagator Setting
et0 = SPICE.utc2et("2024-01-24T17:00:00")
x0 = SPICE.spkez(301, et0, "J2000", "NONE", 0)[1]
tspan = (et0, et0+1000.0)
prob = ODEProblem(eom_nbp!, x0, tspan, (;list_bodies=[10, 399], u=[1.0e-6, 0.0, 0.0]))

# Solve
sol = solve(prob, Vern7(), reltol=1e-15, abstol=1e-15)

retcode: Success
Interpolation: specialized 7th order lazy interpolation
t: 30-element Vector{Float64}:
 7.593876691845803e8
 7.593876692166233e8
 7.593876692823937e8
 7.593876694053108e8
 7.593876696423544e8
 7.593876701591945e8
 7.593876709545634e8
 7.593876721045911e8
 7.593876737267021e8
 7.593876759515848e8
 7.593876790701351e8
 7.593876840568316e8
 7.593876908693898e8
 ⋮
 7.593877997043788e8
 7.593878301470097e8
 7.593878750273749e8
 7.593879203540468e8
 7.593879816758559e8
 7.593880438417637e8
 7.593881208450013e8
 7.59388225085852e8
 7.593883350725138e8
 7.593884588750632e8
 7.593886062774043e8
 7.593886691845803e8
u: 30-element Vector{Vector{Float64}}:
 [-8.337281530069983e7, 1.1208039905539091e8, 4.86573716433273e7, -26.13075980960512, -15.653048739955954, -6.788327413072386]
 [-8.337281613800861e7, 1.1208039855381976e8, 4.8657371425808705e7, -26.13075963869821, -15.653048953965252, -6.7883275135352115]
 [-8.337281785663861e7, 1.1208039752431276e8, 4.865737097933781e7, -26.13